In [1]:
import os, sys
HOME = os.environ['HOME']  # change if necessary
sys.path.append(f'{HOME}/Finite-groups/src')

In [2]:
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
from jaxtyping import Float
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
import pandas as pd
from typing import Union
from einops import repeat
from huggingface_hub import snapshot_download
from huggingface_hub.utils import disable_progress_bars


from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from irrep_bounds import *
from bound_utils import *
%load_ext autoreload
%autoreload 2

Syntax warning: Unbound global variable in /usr/share/gap/pkg/browse/PackageIn\
fo.g:73
  if not IsKernelExtensionAvailable("Browse", "ncurses") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/edim/PackageInfo\
.g:60
  if not IsKernelExtensionAvailable("EDIM","ediv") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^


In [ ]:
group = gapS(5)
irreps = group.get_real_irreps()
subgroups = group.get_subgroups_idx(conjugates=False)

Computing subgroups from gap_repr


Computing elements: 100%|██████████| 17/17 [00:00<00:00, 6276.69it/s]


In [12]:
irrep_bases = dict()
for name, irrep in irreps.items():
    irrep = einops.rearrange(irrep, 'N d1 d2 -> N (d1 d2)')
    U, S, V = t.svd(irrep)
    nonzero = S > 1e-5
    irrep_bases[name] = U[:,nonzero]

In [8]:
subgroups.keys()

dict_keys(['1', 'S5', 'C2_0', 'C2_1', 'C3_0', 'C2xC2_0', 'C2xC2_1', 'C4_0', 'C5_0', 'C6_0', 'S3_0', 'S3_1', 'D8_0', 'D10_0', 'A4_0', 'D12_0', 'C5:C4_0', 'S4_0', 'A5_0'])

In [17]:
for name in sorted(subgroups.keys(), key=lambda x: len(subgroups[x])):
    print(name, len(subgroups[name]))
    subgroup = subgroups[name]
    perm = group.get_permutation_idx(subgroup)
    decomp = set()
    for _ in range(100):
        A = t.randn(perm.shape[-1], perm.shape[-1])
        proj = einops.einsum(perm, A, 'group d1 d2, d1 d2 -> group')
        for name, basis in irrep_bases.items():
            if (basis.H @ proj).norm(dim=0)**2 / (proj.norm(dim=0)**2 + 1e-8) > 1e-2:
                decomp.add(name)
    print(decomp)
    print()

1 1
{'5d-0', '4d-0', '5d-1', '4d-1', '6d-0', '1d-1', '1d-0'}

C2_0 2
{'5d-0', '4d-0', '5d-1', '4d-1', '6d-0', '1d-1'}

C2_1 2
{'5d-0', '4d-0', '5d-1', '4d-1', '6d-0', '1d-1', '1d-0'}

C3_0 3
{'5d-0', '4d-0', '5d-1', '4d-1', '6d-0', '1d-1', '1d-0'}

C2xC2_0 4
{'5d-0', '4d-0', '5d-1', '4d-1', '1d-1', '1d-0'}

C2xC2_1 4
{'5d-0', '5d-1', '4d-1', '6d-0', '1d-1'}

C4_0 4
{'5d-0', '4d-0', '5d-1', '4d-1', '6d-0', '1d-1'}

C5_0 5
{'5d-0', '5d-1', '6d-0', '1d-1', '1d-0'}

C6_0 6
{'4d-0', '5d-1', '4d-1', '6d-0', '1d-1'}

S3_0 6
{'5d-1', '4d-1', '6d-0', '1d-1'}

S3_1 6
{'5d-0', '4d-0', '5d-1', '4d-1', '1d-1', '1d-0'}

D8_0 8
{'5d-1', '5d-0', '4d-1', '1d-1'}

D10_0 10
{'5d-1', '5d-0', '1d-1', '1d-0'}

A4_0 12
{'4d-1', '4d-0', '1d-0', '1d-1'}

D12_0 12
{'5d-1', '4d-1', '1d-1'}

C5:C4_0 20
{'5d-0', '1d-1'}

S4_0 24
{'4d-1', '1d-1'}

A5_0 60
{'1d-1', '1d-0'}

S5 120
{'1d-1'}

